# Sign Recognizer CNN

This notebook will create the sign recognizer model

In [1]:
%matplotlib inline
import seaborn as sns

import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import text

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

from tqdm import tqdm

import glob, os

Using TensorFlow backend.


In [2]:
img_rows, img_cols = 96, 128
in_shape = (img_rows, img_cols, 3)
batch_size = 256
nb_classes = 2
nb_epoch = 1500

In [3]:
os.chdir("/home/nathan/olin/spring2017/sign_follower/scripts/data/train2")

In [4]:
train_datagen = ImageDataGenerator(
                rotation_range=10,
                width_shift_range=0.02,
                height_shift_range=0.02,
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                vertical_flip = False,
                horizontal_flip=False)

In [5]:
train_generator = train_datagen.flow_from_directory(
        '/home/nathan/olin/spring2017/sign_follower/scripts/data/train2',  # this is the target directory
        target_size=(img_rows, img_cols),  # all images will be resized to 96x128
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 31382 images belonging to 2 classes.


In [6]:
train_generator.class_indices

{'room': 0, 'sign': 1}

# Model

## Define model

In [7]:
model = Sequential()

## Add convolutional layers

Nothing special, just 3 sequential 64 3x3 conv layers

In [8]:
model.add(Convolution2D(64, 3, 3, border_mode='valid', input_shape=in_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

## Add fully connected layers

Single fully connected layer

In [9]:
model.add(Flatten())
model.add(Dense(192))
model.add(Activation('relu'))
# model.add(Dropout(0.5)) # helps prevent overfitting


model.add(Dense(nb_classes))
model.add(Activation('softmax'))

## Compile model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
history = model.fit_generator(
        train_generator,
        samples_per_epoch=batch_size,
        nb_epoch=nb_epoch,
        verbose=True)

Epoch 1/1500
256/256 [==============================] - 2s - loss: 0.7332 - acc: 0.0664
Epoch 2/1500
256/256 [==============================] - 0s - loss: 0.2507 - acc: 0.9805
Epoch 3/1500
256/256 [==============================] - 0s - loss: 0.0696 - acc: 0.9961
Epoch 4/1500
256/256 [==============================] - 0s - loss: 0.3744 - acc: 0.9766
Epoch 5/1500
256/256 [==============================] - 0s - loss: 0.3183 - acc: 0.9805
Epoch 6/1500
256/256 [==============================] - 0s - loss: 0.0639 - acc: 0.9961
Epoch 7/1500
256/256 [==============================] - 0s - loss: 0.2519 - acc: 0.9844
Epoch 8/1500
256/256 [==============================] - 0s - loss: 0.1889 - acc: 0.9883
Epoch 9/1500
256/256 [==============================] - 0s - loss: 0.4407 - acc: 0.9727
Epoch 10/1500
256/256 [==============================] - 0s - loss: 0.2518 - acc: 0.9844
Epoch 11/1500
256/256 [==============================] - 0s - loss: 0.4407 - acc: 0.9727
Epoch 12/1500
256/256 [=======

Old weights:  
1_maybeGotLucky.h5  
2_betterDataset.h5  
3_removedFlips.h5

In [ ]:
model.save_weights('3_removedFlips.h5')

In [ ]:
model.load_weights('3_removedFlips.h5')

# Load test data

In [ ]:
from PIL import Image

In [ ]:
os.chdir("/home/nathan/olin/spring2017/sign_follower/scripts/data/valid2")

In [ ]:
test_data = np.asarray([np.asarray(Image.open(file)) for file in glob.glob("*.png")])

In [ ]:
# for file in tqdm(glob.glob("*.png")):
#     img  = Image.open(file)
#     data = np.asarray(img)
#     try:
#         test_data = np.concatenate((test_data, data), axis=0)
#     except:
#         test_data = np.asarray(data)

In [ ]:
test_data.shape

In [ ]:
prediction = model.predict(test_data, verbose=True)

In [ ]:
np.sum(prediction, axis=0)

In [ ]:
x = 100
nb = 0 
for pred, file in zip(prediction, glob.glob("*.png")):
    
    if pred[1] > .9:
        Image.open(file).show()
        nb += 1
    if nb == x:
        break